<a href="https://colab.research.google.com/github/AbinSibi466/NNDL/blob/main/Abin_202_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
afdadfadfsadfdsafa